In [ ]:
! pip install -q chromadb
# llama-cpp-python version greater or equal to 0.2.25 is required. 
# You will encounter a "NULL pointer access" error if you have an older version installed.
! pip install -q llama-cpp-python>=0.2.25


In [ ]:
from langchain.embeddings import LlamaCppEmbeddings

embeddings = LlamaCppEmbeddings(
    model_path="[PATH_TO_YOUR_MODEL]/llama.cpp/models/llama-2-7b-chat/ggml-model-q5_0.gguf",
    verbose=False,
    )


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

transcript_docs = TextLoader('../sample-data/gartner-events/Session 1 - Opening Keynote.txt').load()
text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=25,
    length_function=len,
    add_start_index=True
)

In [ ]:
from langchain.vectorstores import Chroma

documents = text_spliter.split_documents(transcript_docs)
vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
query = "What the current trend in AI?"
docs = vectorstore.similarity_search(query, k=3)
print(docs[2].page_content)


In [ ]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

llm  = LlamaCpp(
    model_path="[PATH_TO_YOUR_MODEL]/llama.cpp/models/llama-2-7b-chat/ggml-model-q5_0.gguf",
    verbose=False,
    temperature=0.25,
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True
)


prompt_template = """
[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> 
Question: {question} 
Context: {context} 
Answer: [/INST]
"""


prompt = PromptTemplate.from_template(prompt_template)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=vectorstore.as_retriever(), 
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
from langchain.globals import set_debug

set_debug(True)

question = "What the current trend in AI?"
result = qa_chain({"query": question})
result["result"]


In [ ]:
question = "How will AI affect education?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "How will AI affect pharma industry?"
result = qa_chain({"query": question})
result["result"]